In [89]:
# depedencies

import pandas as pd
import sklearn as sk
from sklearn import svm
from sklearn.decomposition import PCA
import numpy
import random
import matplotlib.pyplot as plot
from deap.algorithms import eaSimple
from deap import base, creator, tools
from sklearn.datasets import load_wine
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
# reading dataset

patient_data = pd.read_csv('data.csv')
patient_data = patient_data.iloc[: , 1:]
print(patient_data.shape)

In [ ]:
# implementing various classification function
y = patient_data.iloc[:,23]
X = patient_data.iloc[:,:23]
colList = list(X.columns)
colList[7], colList[1] =  colList[1], colList[7]
X = X[colList]
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

print("Features:\n")
print(*X, sep='\n')

def performSVM(x_train, x_test, y_train, y_test):
    SVM = svm.LinearSVC(dual=False)
    SVM.fit(x_train, y_train)
    print("Accuracy on SVM:")
    score = SVM.score(x_test, y_test)
    print(score)

def performLGR(x_train, x_test, y_train, y_test):
    lgr = LogisticRegression(max_iter=5000)
    lgr.fit(x_train, y_train)
    # predictions = lgr.predict(x_test)
    score = lgr.score(x_test, y_test)
    print("Accuracy on LR:")
    print(score)

def performKnn(x_train, x_test, y_train, y_test):
    knn = KNeighborsClassifier(n_neighbors=3)
    knn.fit(x_train,y_train)
    score = knn.score(x_test, y_test)
    print("Accuracy on Knn:")
    print(score)
    return score

def performKnnPred(x_train, x_test, y_train, y_test):
    knn = KNeighborsClassifier(n_neighbors=3)
    knn.fit(x_train,y_train)
    score = knn.score(x_test, y_test)
    preds = knn.predict(x_test)
    print("Accuracy on Knn:")
    print(score)
    return score, preds

def performANN(x_train, x_test, y_train, y_test):
    model = keras.Sequential(
        [
            layers.Dense(23, activation="relu", name="layer1"),
            layers.Dense(10, activation="relu", name="layer2"),
            layers.Dense(3, activation="softmax",name="layer3"),
        ]
    )
    model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['sparse_categorical_accuracy'])
    model.fit(x_train, y_train, epochs=50)
    model.evaluate(x_test, y_test)

# function to remove the unselected features from the given array from GA algorithm
def reduceFeatures(features, selectionArr):
    selected_features = features
    for i in selectionArr:
        if i == 0:
            selected_features = selected_features.drop(selected_features.columns[[i]], axis=1) 
    return selected_features


In [ ]:
features = X
target = y

# split the data into train and test
x_train, x_test, y_train, y_test = train_test_split(
    features, target, test_size=0.2, stratify=target
)

model = LogisticRegression(max_iter=5000)

def fitness_score(individual):
    column_support = pd.Series(individual).astype(bool)
    global x_train, y_train, x_test, y_test, model
    
    x_train_ = x_train[x_train.columns[column_support]]
    x_test_ = x_test[x_test.columns[column_support]]

    model.fit(x_train_, y_train)
    y_pred = model.predict(x_test_)
    score = accuracy_score(y_test, y_pred)
    
    return score,

n_genes = x_train.shape[1]
n_generations = 10
n_population = 20
crossover_probability = 0.7
mutation_probability = 0.2

def setup_toolbox():
    # container for individuals
    creator.create('FitnessMax', base.Fitness, weights=(1.0,))
    creator.create('Individual', list, fitness=creator.FitnessMax)

    toolbox = base.Toolbox()
    toolbox.register(
        'individual_generator_function',
        random.randint, 0, 1 
    )
    # method to populate individual
    toolbox.register(
        'individual_generator',
        tools.initRepeat,
        creator.Individual,
        toolbox.individual_generator_function,
        n_genes
    )
    # method to create population
    toolbox.register(
        'population_generator',
        tools.initRepeat,
        list,
        toolbox.individual_generator
    )
    # fitness calculation
    toolbox.register(
        'evaluate', fitness_score
    )
    # selection
    toolbox.register(
        'select', tools.selRoulette
    )
    # crossover
    toolbox.register('mate', tools.cxOnePoint)
    # mutation
    toolbox.register(
        'mutate',
        tools.mutFlipBit,
        indpb=mutation_probability
    )
    return toolbox

# setup deap toolbox
toolbox = setup_toolbox()

# create a population
population = toolbox.population_generator(n_population)

# A simple evolutionary algorithm
final_feature_population, logbook = eaSimple(
    population, 
    toolbox, 
    crossover_probability, 
    mutation_probability, 
    n_generations
)


In [ ]:
least_amount_feat = []
feat_count = 20

for n in final_feature_population:
    # score = compute_fitness_score(n)
    selected_feats = reduceFeatures(features, n)
    x_train, x_test, y_train, y_test = train_test_split(selected_feats, target, test_size=0.2)
    score = performKnn(x_train, x_test, y_train, y_test)
    print(n)
    currrent_feat_count = list(n).count(1)
    print(currrent_feat_count)
    print(score)
    if score > 0.9:
        if currrent_feat_count < feat_count:
            least_amount_feat = n
            feat_count = currrent_feat_count


In [ ]:
# picking the reduced features with best fitness score from round 1
print("Selected feature array")
print(least_amount_feat)
count = list(least_amount_feat).count(1)
print(count)

# picking selected features from the original feature table
selected_feats = reduceFeatures(features, least_amount_feat)
x_train, x_test, y_train, y_test = train_test_split(selected_feats, target, test_size=0.2)

print("Reduced Features (Round 1):\n")
print(*selected_feats, sep='\n')

# kNN on reduced features
score, preds = performKnnPred(x_train, x_test, y_train, y_test)
print(confusion_matrix(y_test, preds))

# SVM on reduced features
performSVM(x_train, x_test, y_train, y_test)

# LGR on reduced features
performLGR(x_train, x_test, y_train, y_test)

# ANN on reduced features
performANN(x_train, x_test, y_train, y_test)

In [ ]:
features = selected_feats
target = y

# split the data into train and test
x_train, x_test, y_train, y_test = train_test_split(
    features, target, test_size=0.2, stratify=target
)

model = LogisticRegression(max_iter=5000)

def fitness_function(individual):
    """
    Select the features from the individual, train
    and compute the accuracy_score.
    
    Example:
    individual = [0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1]
    The 1 represents the presence of features and
    0 represents the absence of features
    
    """
    column_support = pd.Series(individual).astype(bool)
    global x_train, y_train, x_test, y_test, model
    
    x_train_ = x_train[x_train.columns[column_support]]
    x_test_ = x_test[x_test.columns[column_support]]

    model.fit(x_train_, y_train)
    y_pred = model.predict(x_test_)
    score = accuracy_score(y_test, y_pred)
    
    return score,

n_genes = x_train.shape[1]
n_generations = 10
n_population = 8
crossover_probab = 0.7
mutation_probab = 0.2

def setup_toolbox():
    # container for individuals
    creator.create('FitnessMax', base.Fitness, weights=(1.0,))
    creator.create('Individual', list, fitness=creator.FitnessMax)

    toolbox = base.Toolbox()
    toolbox.register(
        'individual_generator_function',
        random.randint, 0, 1 
    )
    # method to populate individual
    toolbox.register(
        'individual_generator',
        tools.initRepeat,
        creator.Individual,
        toolbox.individual_generator_function,
        n_genes
    )
    # method to create population
    toolbox.register(
        'population_generator',
        tools.initRepeat,
        list,
        toolbox.individual_generator
    )
    # fitness score calculation
    toolbox.register(
        'evaluate', fitness_function
    )
    # selection
    toolbox.register(
        'select', tools.selRoulette
    )
    # crossover
    toolbox.register('mate', tools.cxOnePoint)
    # mutation
    toolbox.register(
        'mutate',
        tools.mutFlipBit,
        indpb=mutation_probab
    )
    return toolbox

# setup deap toolbox
toolbox = setup_toolbox()

# create a population
population = toolbox.population_generator(n_population)

# A simple evolutionary algorithm
final_feature_population, logbook = eaSimple(
    population, 
    toolbox, 
    crossover_probab, 
    mutation_probab, 
    n_generations
)

In [ ]:
# picking the best feature set from the round 2 of GA with feature count equal to 5
least_amount_feat = []
feat_count = 8

for feature in final_feature_population:
    # score = compute_fitness_score(n)
    new_selected_feats = reduceFeatures(selected_feats, feature)
    x_train, x_test, y_train, y_test = train_test_split(new_selected_feats, target, test_size=0.2)
    score = performKnn(x_train, x_test, y_train, y_test)
    print(feature)
    currrent_feat_count = list(feature).count(1)
    print(currrent_feat_count)
    print(score)
    if score > 0.9:
        if currrent_feat_count == 5:
            least_amount_feat = feature

In [ ]:
# calculating the accuracy and performance metrices on features selected from second round of GA
print("Selected feature array")
print(least_amount_feat)
count = list(least_amount_feat).count(1)
print(count)

# picking selected features from the original feature table
selected_feats = reduceFeatures(features, least_amount_feat)
x_train, x_test, y_train, y_test = train_test_split(selected_feats, target, test_size=0.2)

print("Reduced Features (Round 2):\n")
print(*selected_feats, sep='\n')

# kNN on reduced features
score, preds = performKnnPred(x_train, x_test, y_train, y_test)
print(confusion_matrix(y_test, preds))

print(f1_score(y_test, preds, average="macro"))
print(precision_score(y_test, preds, average="macro"))
print(recall_score(y_test, preds, average="macro")) 

# SVM on reduced features
performSVM(x_train, x_test, y_train, y_test)

# LGR on reduced features
performLGR(x_train, x_test, y_train, y_test)